# Разработка API

In [43]:
import pickle
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
import pandas as pd 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import warnings
warnings.filterwarnings("ignore")
import re
import numpy as np
import pdfplumber
import pymorphy2
import pdfplumber
import pymorphy2
from fpdf import FPDF

In [58]:
class API():
    """Class Api Представляет собо совокупность методов, с помощью которых вы можете определить категорию вашей жалобы или обращения.\nТакже для разработчиков существует коррекция результатов и сохранения их для послудующего обучения
    \nМетоды:
    print_res(self)
    load_model(self, filename = "model.pkl")
    cleaning(self)
    make_pred(self, p_num=1)
    correction(self)
    input_words_manually(self)
    load_correction_dataset(self)
    create_correction_dataset(self)
    save_results(self)
    
    """
    
    # переменные класса
    words = ""
    model = ""
    R = []
    data = ""
    comp_cat=[]
    
    def __init__(self):
        pass
    
    def print_res(self):
        """printing results"""
        for i in self.R:
            print(i)

    # функции

    def load_model(self, filename = "model.pkl"):
        """Функция загрузки, по умолчанию принемает значение filename = "model.pkl, если у вас есть другая модель вы можете воспользоваться ею"""
        with open(filename, 'rb') as file:
                model = pickle.load(file)
        self.model = model 
    
        
    def cleaning(self):
        """Очистка слов от символов, стопслов и лемматизация, используются модули re, pymorph2"""
        words = re.sub('[^a-zA-zа-яА-Я]', ' ', self.words).lower().strip()
        stopwords = ["тд","c","а","алло","без","белый","близко","более","больше","большой","будем","будет","будете","будешь","будто","буду","будут","будь","бы","бывает","бывь","был","была","были","было","быть","в","важная","важное","важные","важный","вам","вами","вас","ваш","ваша","ваше","ваши","вверх","вдали","вдруг","ведь","везде","вернуться","весь","вечер","взгляд","взять","вид","видел","видеть","вместе","вне","вниз","внизу","во","вода","война","вокруг","вон","вообще","вопрос","восемнадцатый","восемнадцать","восемь","восьмой","вот","впрочем","времени","время","все","все еще","всегда","всего","всем","всеми","всему","всех","всею","всю","всюду","вся","всё","второй","вы","выйти","г","где","главный","глаз","говорил","говорит","говорить","год","года","году","голова","голос","город","да","давать","давно","даже","далекий","далеко","дальше","даром","дать","два","двадцатый","двадцать","две","двенадцатый","двенадцать","дверь","двух","девятнадцатый","девятнадцать","девятый","девять","действительно","дел","делал","делать","делаю","дело","день","деньги","десятый","десять","для","до","довольно","долго","должен","должно","должный","дом","дорога","друг","другая","другие","других","друго","другое","другой","думать","душа","е","его","ее","ей","ему","если","есть","еще","ещё","ею","её","ж","ждать","же","жена","женщина","жизнь","жить","за","занят","занята","занято","заняты","затем","зато","зачем","здесь","земля","знать","значит","значить","и","иди","идти","из","или","им","имеет","имел","именно","иметь","ими","имя","иногда","их","к","каждая","каждое","каждые","каждый","кажется","казаться","как","какая","какой","кем","книга","когда","кого","ком","комната","кому","конец","конечно","которая","которого","которой","которые","который","которых","кроме","кругом","кто","куда","лежать","лет","ли","лицо","лишь","лучше","любить","люди","м","маленький","мало","мать","машина","между","меля","менее","меньше","меня","место","миллионов","мимо","минута","мир","мира","мне","много","многочисленная","многочисленное","многочисленные","многочисленный","мной","мною","мог","могу","могут","мож","может","может быть","можно","можхо","мои","мой","мор","москва","мочь","моя","моё","мы","на","наверху","над","надо","назад","наиболее","найти","наконец","нам","нами","народ","нас","начала","начать","наш","наша","наше","наши","не","него","недавно","недалеко","нее","ней","некоторый","нельзя","нем","немного","нему","непрерывно","нередко","несколько","нет","нею","неё","ни","нибудь","ниже","низко","никакой","никогда","никто","никуда","ним","ними","них","ничего","ничто","но","новый","нога","ночь","ну","нужно","нужный","нх","о","об","оба","обычно","один","одиннадцатый","одиннадцать","однажды","однако","одного","одной","оказаться","окно","около","он","она","они","оно","опять","особенно","остаться","от","ответить","отец","откуда","отовсюду","отсюда","очень","первый","перед","писать","плечо","по","под","подойди","подумать","пожалуйста","позже","пойти","пока","пол","получить","помнить","понимать","понять","пор","пора","после","последний","посмотреть","посреди","потом","потому","почему","почти","правда","прекрасно","при","про","просто","против","процентов","путь","пятнадцатый","пятнадцать","пятый","пять","работа","работать","раз","разве","рано","раньше","ребенок","решить","россия","рука","русский","ряд","рядом","с","с кем","сам","сама","сами","самим","самими","самих","само","самого","самой","самом","самому","саму","самый","свет","свое","своего","своей","свои","своих","свой","свою","сделать","сеаой","себе","себя","сегодня","седьмой","сейчас","семнадцатый","семнадцать","семь","сидеть","сила","сих","сказал","сказала","сказать","сколько","слишком","слово","случай","смотреть","сначала","снова","со","собой","собою","советский","совсем","спасибо","спросить","сразу","стал","старый","стать","стол","сторона","стоять","страна","суть","считать","т","та","так","такая","также","таки","такие","такое","такой","там","твои","твой","твоя","твоё","те","тебе","тебя","тем","теми","теперь","тех","то","тобой","тобою","товарищ","тогда","того","тоже","только","том","тому","тот","тою","третий","три","тринадцатый","тринадцать","ту","туда","тут","ты","тысяч","у","увидеть","уж","уже","улица","уметь","утро","хороший","хорошо","хотел бы","хотеть","хоть","хотя","хочешь","час","часто","часть","чаще","чего","человек","чем","чему","через","четвертый","четыре","четырнадцатый","четырнадцать","что","чтоб","чтобы","чуть","шестнадцатый","шестнадцать","шестой","шесть","эта","эти","этим","этими","этих","это","этого","этой","этом","этому","этот","эту","я","являюсь"]
        words = ' '.join([word for word in words.split() if word not in (stopwords)])
        morph = pymorphy2.MorphAnalyzer()
        words = " ".join(morph.parse(word)[0].normal_form for word in words.split())
        self.words = words
        
    def make_pred(self, p_num=1):
        """Функция предикта позволяющая определить одну или несколько компетенций, один аргумент p_num - кол-во предсказания"""
        self.comp_cat=[]
        self.R=[]
        if p_num==1:
            pred = self.model.predict([self.words])
            self.R.append(pred)
        else:
            allpreds = self.model.predict_proba([self.words]).reshape(-1)        
            for i in allpreds.argsort()[-p_num:][::-1]:
                self.R.append(self.model.classes_[i])
                
    
    #коррекция результатов

    def correction(self):
        """Функция коррекции результата если пользователю не понравился результат работы программы, результат сохраняется в файл forFutureFitting.csv"""
        try:
            self.load_correction_dataset()
        except Exception:
            self.create_correction_dataset()
        
        tmp = {}
        tmp["category"] = input('>>> Введите названия категории \n>>> ')
        tmp["words"] = self.words
        
        self.data = self.data.append(tmp, ignore_index=True)
        self.data.to_csv("forFutureFitting.csv")
        
                
    def input_words_manually(self):
        """Ручной ввод слов"""
        self.words = input(">>> Введите оприсание своих навыков: \n>>>")
        
    def load_correction_dataset(self):
        """Загрузка коррекционного датасета"""
        self.data = pd.read_csv("forFutureFitting.csv", index_col=0)
        
    def create_correction_dataset(self):
        """Создание коррекционного датасета"""
        pd.DataFrame(columns=["words","category"]).to_csv("forFutureFitting.csv")
        self.data = pd.DataFrame(columns=["words","category"])
    
    def save_results(self, filename):
        """Функция сохранения результата в pdf файл, аргумет filename - имя файла, используется модуль fpdf"""
        text = "Категория: "+ str(self.R[0]) +"\nОбращение/жалоба: \n"  + str(self.words)

        
        
        pdf = FPDF()
        pdf.add_page()

        pdf.add_font('arial', '', 'arial.ttf', uni=True)
        pdf.set_font('arial', '', 14)
        
        for txt in text.split('\n'):
            pdf.write(8, txt)
            pdf.ln(8)
        pdf.output(filename)

In [32]:
api = API()

In [33]:
print(api.__doc__)

Class Api Представляет собо совокупность методов, с помощью которых вы можете определить категорию вашей жалобы или обращения.
Также для разработчиков существует коррекция результатов и сохранения их для послудующего обучения
    
Методы:
    print_res(self)
    load_model(self, filename = "model.pkl")
    cleaning(self)
    make_pred(self, p_num=1)
    correction(self)
    input_words_manually(self)
    load_correction_dataset(self)
    create_correction_dataset(self)
    save_results(self)
    
    


In [34]:
api.load_model()

In [35]:
api.input_words_manually()

>>> Введите оприсание своих навыков: 
>>>к какой категории мне обратиться если в моем доме не исправен лифт


In [56]:
api.cleaning()

In [36]:
api.make_pred(p_num=1)

In [57]:
api.print_res()

['Содержание МКД']


In [49]:
api.create_correction_dataset()

In [50]:
api.correction()

>>> Введите названия категории 
>>> фасад


In [51]:
api.data

,words,category
0,к какой категории мне обратиться если в моем д...,фасад


In [59]:
api.save_results("output.pdf")

In [30]:
api.R

['Нарушение правил пользования общим имуществом',
 'Центральное отопление',
 'Фасад']